# Problemas de Busca em IA

O campo de estudo da Inteligência Artificial (IA) fica cada vez mais famoso a cada notícia que aborda o tema. Tarefas incríveis sendo realizadas por máquinas que antes eram apenas fruto da nossa imaginação.

Entretanto, é sempre bom atentarmos para os fundamentos. Um deles é o fato de que esses sistemas ou agentes inteligentes em algum momento passam por um processo de busca de uma solução dentro de um espaço de estados ou valores, tal busca sendo performada por algoritmos próprios.

A diversidade destes algoritmos é enorme. Mas como estamos falando de fundamentos, vejamos alguns que são fundamentais. São eles os algoritmos de:
- Busca em Largura;
- Busca em Profundidade;
- Busca Gulosa.

Os dois primeiros fazem parte de uma categoria chamada "busca cega", pois ela acontece sem nenhuma informação sobre o problema. A última por sua vez está na categoria de "busca heurística", pois leva em consideração alguma informação, diferente da busca cega.

Para aplicar os algoritmos, vamos abordar um problema real: a solução do quebra-cabeça de 8 peças.

## Quebra-Cabeça de 8 Peças

Um quebra-cabeça de 8 peças é um jogo comum como ilustrado na figura abaixo:

![quebra-cabeca.png](quebra-cabeca.png)

A partir de um estado inicial, o objetivo é alcançar outro com o mínimo de movimentos possível. Inclusive, para esta exemplificação, os estados inicial e final desejados serão estes:

![estados.png](estados.png)

Vamos ao código!

## Implementação

A seguir, vejamos como modelar este problema e sua solução para os 3 algoritmos de busca citados, sendo que as heurísticas para a busca gulosa serão:
- Número de peças fora do lugar;
- Distância Manhattan entre as peças.

### Importação de Bibliotecas

Para este código, vamos precisar somente das seguintes bibliotecas:

In [1]:
import time
from collections import deque

### Representação do Problema

Uma das grandes dificuldades de solucionar problemas da vida real de forma computacional é como fazer uma boa representação do problema. 

Vamos começar tratando sobre como representar cada estado do tabuleiro.

Uma forma inteligente é usar um vetor de 9 posições, onde cada posição é uma peça do tabuleiro, imaginando que a cada 3 peças aconteça uma quebra de linha.

Por exemplo, os estados inicial e final poderiam ser escritos como:

In [2]:
estado_inicial = [0, 1, 2, 7, 8, 3, 6, 5, 4]
solucao_desejada = [1, 2, 3, 8, 0, 4, 7, 6, 5]

Além disso, vamos precisar de algumas outras variáveis globais para que o código funcione:
- `vertice_solucao`: armazena o estado final com a solução do problema;
- `total_pecas_quebra_cabeca`: acho que o nome é autoexplicativo;
- `tamanho_lado_quebra_cabeca`: ou seja, se tratando de um tabuleiro quadrangular, é a raiz quadrada do total de peças do quebra-cabeça;
- `total_vertices_visitados`: por quantos nós passou o algoritmo até encontrar uma solução;
- `profundidade_maxima`: o quão profundo chegou na árvore de busca;
- `largura_maxima`: similar, mas referente à largura.

In [3]:
total_pecas_quebra_cabeca = len(estado_inicial)
tamanho_lado_quebra_cabeca = int(total_pecas_quebra_cabeca ** 0.5)
total_vertices_visitados = 0
profundidade_maxima = 0
largura_maxima = 0

Agora, vamos definir uma classe que representa um possível estado do quebra-cabeça.

In [4]:
class EstadoQuebraCabeca:
    def __init__(self, estado, pai, movimento, profundidade, custo):
        self.estado = estado
        self.pai = pai
        self.movimento = movimento
        self.profundidade = profundidade
        self.custo = custo
        if self.estado:
            self.mapeamento = "".join(str(numeroEstado)
                                      for numeroEstado in self.estado)

Nesta classe, temos variáveis para armazenar o estado atual em que está o quebra-cabeça (um vetor como definido anteriormente), quem é seu antecessor ("pai"), qual movimento anterior de peça que levou a essa configuração, o quão profundo ele está na árvore de busca e o custo para chegar até esse ponto.

Agora, vamos definir uma função para simular um movimento de peça:

In [5]:
def movimento(estado, direcao):
    global tamanho_lado_quebra_cabeca, total_pecas_quebra_cabeca

    novo_estado = estado[:]
    indice = novo_estado.index(0)

    # Para cima
    if direcao == 1:
        if indice not in range(0, tamanho_lado_quebra_cabeca):
            temp = novo_estado[indice - tamanho_lado_quebra_cabeca]
            novo_estado[indice -
                        tamanho_lado_quebra_cabeca] = novo_estado[indice]
            novo_estado[indice] = temp

            return novo_estado
        else:
            return None

    # Para baixo
    if direcao == 2:
        if indice not in range(
            total_pecas_quebra_cabeca - tamanho_lado_quebra_cabeca, total_pecas_quebra_cabeca
        ):

            temp = novo_estado[indice + tamanho_lado_quebra_cabeca]
            novo_estado[indice +
                        tamanho_lado_quebra_cabeca] = novo_estado[indice]
            novo_estado[indice] = temp

            return novo_estado
        else:
            return None

    # Para esquerda
    if direcao == tamanho_lado_quebra_cabeca:
        if indice not in range(0, total_pecas_quebra_cabeca, tamanho_lado_quebra_cabeca):
            temp = novo_estado[indice - 1]
            novo_estado[indice - 1] = novo_estado[indice]
            novo_estado[indice] = temp

            return novo_estado
        else:
            return None

    # Para direita
    if direcao == 4:
        if indice not in range(
            tamanho_lado_quebra_cabeca - 1, total_pecas_quebra_cabeca, tamanho_lado_quebra_cabeca
        ):
            temp = novo_estado[indice + 1]
            novo_estado[indice + 1] = novo_estado[indice]
            novo_estado[indice] = temp

            return novo_estado
        else:
            return None

É interessante ter em mente que, ao invés de pensar que uma peça está se movimento para um estado vazio (como é comum de se jogar), na verdade é o estado vazio que se move para o lugar de uma peça.

Vale lembrar que nesta implementação, o espaço vazio do tabuleiro é representado por um 0.

Pensando dessa forma, tem-se uma "peça" que sempre vai se mover, que na verdade é o próprio estado vazio.

Portanto, vale apenas verificar se este está em alguma das extremidades do tabuleiro no momento de tentar um movimento. Se estiver, não é possível movê-lo; se não estiver, o movimento é possível.

Agora, vamos implementar uma forma de expandir a árvore de busca e encontrar todos os próximos movimentos possíveis a partir de um determinado estado:

In [6]:
def sub_vertices(vertice):
    global total_vertices_visitados
    total_vertices_visitados = total_vertices_visitados + 1

    proximos_caminhos_possiveis = []
    proximos_caminhos_possiveis.append(
        EstadoQuebraCabeca(
            movimento(vertice.estado, 1),
            vertice,
            1,
            vertice.profundidade + 1,
            vertice.custo + 1,
        )
    )
    proximos_caminhos_possiveis.append(
        EstadoQuebraCabeca(
            movimento(vertice.estado, 2),
            vertice,
            2,
            vertice.profundidade + 1,
            vertice.custo + 1,
        )
    )
    proximos_caminhos_possiveis.append(
        EstadoQuebraCabeca(
            movimento(vertice.estado, 3),
            vertice,
            3,
            vertice.profundidade + 1,
            vertice.custo + 1,
        )
    )
    proximos_caminhos_possiveis.append(
        EstadoQuebraCabeca(
            movimento(vertice.estado, 4),
            vertice,
            4,
            vertice.profundidade + 1,
            vertice.custo + 1,
        )
    )
    vertices = []
    for caminhos_realmente_possiveis in proximos_caminhos_possiveis:
        if caminhos_realmente_possiveis.estado != None:
            vertices.append(caminhos_realmente_possiveis)

    return vertices

A ideia é simples: é só testar todos os quatro movimentos, ver quais são possíveis e retorná-los ao final.

Com essas funções, já é possível realizar buscas, tendo o problema sido bem definido.

### Busca em Largura

O funcionamento de uma busca em largura pode se assimilar a uma implementação de fila (FIFO). Então, a partir de um estado inicial, busca-se o final respeitando a ordem de entrada e saída dos demais estados explorados nessa estrutura.

Uma implementação possível é assim:

In [7]:
def busca_largura(estado_inicial):
    global largura_maxima, vertice_solucao, profundidade_maxima

    fila = deque([EstadoQuebraCabeca(estado_inicial, None, None, 0, 0)])
    vertice_visitado = set()

    while fila:
        vertice = fila.popleft()
        vertice_visitado.add(vertice.mapeamento)
        if vertice.estado == solucao_desejada:
            vertice_solucao = vertice
            return fila

        caminhos_possiveis = sub_vertices(vertice)
        for caminho in caminhos_possiveis:
            if caminho.mapeamento not in vertice_visitado:
                fila.append(caminho)
                vertice_visitado.add(caminho.mapeamento)
                if caminho.profundidade > profundidade_maxima:
                    profundidade_maxima = profundidade_maxima + 1
        if len(fila) > largura_maxima:
            largura_maxima = len(fila)

Ou seja, enquanto houver elementos na fila, o código é executado e os valores globais são atualizados, até que se encontre uma solução ou não haja mais elementos.

### Busca em Profundidade

Similar à lógica da busca em largura, a busca em profundidade só difere na estrutura de organização dos estados. Ao invés de usar uma fila, utiliza uma pilha (LIFO). Simples assim.

Portanto, a implementação também é bem parecida.

In [8]:
def busca_profundidade(estado_inicial):
    global largura_maxima, vertice_solucao, profundidade_maxima

    vertice_visitado = set()
    pilha = list([EstadoQuebraCabeca(estado_inicial, None, None, 0, 0)])

    while pilha:
        vertice = pilha.pop()
        vertice_visitado.add(vertice.mapeamento)
        if vertice.estado == solucao_desejada:
            vertice_solucao = vertice
            return pilha

        caminhos_possiveis = sub_vertices(vertice)
        for caminho in caminhos_possiveis:
            if caminho.mapeamento not in vertice_visitado:
                pilha.append(caminho)
                vertice_visitado.add(caminho.mapeamento)
                if caminho.profundidade > profundidade_maxima:
                    profundidade_maxima = 1 + profundidade_maxima
        if len(pilha) > largura_maxima:
            largura_maxima = len(pilha)

### Busca Gulosa

Como dito anteriormente, a busca gulosa funciona levando em conta algum conhecimento sobre o problema e os estados atuais, sendo que aqui são consideradas duas heurísticas: número de peças fora do lugar e distância Manhattan.

Para calcular o número de peças fora do lugar, pode fazer assim:

In [9]:
def heuristica_pecas_fora_do_lugar(estado_atual, estado_desejado):
    pecas_fora_do_lugar = 0

    for i in range(len(estado_atual)):
        pecas_fora_do_lugar += 1 if estado_atual[i] != estado_desejado[i] else 0

    return pecas_fora_do_lugar

É só comparar posição a posição dos dois vetores, de estado atual e desejado, e contabilizar a diferença.

A distância Manhatann é um pouco mais complicada:

In [10]:
def heuristica_manhattan(estado_atual, estado_desejado):
    global tamanho_lado_quebra_cabeca, total_pecas_quebra_cabeca

    distancia = sum(
        abs(atual % tamanho_lado_quebra_cabeca - desejado %
            tamanho_lado_quebra_cabeca)
        + abs(atual // tamanho_lado_quebra_cabeca -
              desejado // tamanho_lado_quebra_cabeca)
        for atual, desejado in (
            (estado_atual.index(i), estado_desejado.index(i))
            for i in range(1, total_pecas_quebra_cabeca)
        )
    )

    return distancia

Pensando de maneira matricial, ou seja, nas posições das peças como se fosse em uma matriz, dá pra considerar a soma das diferenças dos módulos entre cada peça na posição atual e desejada com as diferenças dos restos das mesmas peças.

Com isso, podemos implementar a busca gulosa:

In [11]:
def ordena_por_prioridade(lista):
    return lista["prioridade"]

def busca_gulosa(estado_inicial, heuristica):
    global largura_maxima, vertice_solucao, profundidade_maxima

    prioridade = (
        heuristica_pecas_fora_do_lugar(estado_inicial, solucao_desejada)
        if heuristica == "1"
        else heuristica_manhattan(estado_inicial, solucao_desejada)
    )

    vertice_visitado = set()
    lista_ordenada = [
        {
            "prioridade": prioridade,
            "estado": EstadoQuebraCabeca(estado_inicial, None, None, 0, 0),
        }
    ]

    lista_ordenada.sort(key=ordena_por_prioridade)

    while lista_ordenada:
        vertice = lista_ordenada.pop()
        vertice_visitado.add(vertice["estado"].mapeamento)
        if vertice["estado"].estado == solucao_desejada:
            vertice_solucao = vertice["estado"]
            return lista_ordenada[:][-1]

        caminhos_possiveis = sub_vertices(vertice["estado"])

        prioridades_caminhos_possiveis = [
            heuristica_pecas_fora_do_lugar(
                estado_atual.estado, solucao_desejada)
            if heuristica == "1"
            else heuristica_manhattan(estado_atual.estado, solucao_desejada)
            for estado_atual in caminhos_possiveis
        ]

        caminhos_possiveis_ordenados = [
            {"prioridade": prioridade, "estado": estado}
            for prioridade, estado in zip(
                prioridades_caminhos_possiveis, caminhos_possiveis
            )
        ]
        caminhos_possiveis_ordenados.sort(key=ordena_por_prioridade)

        for caminho in caminhos_possiveis_ordenados:
            if caminho["estado"].mapeamento not in vertice_visitado:
                lista_ordenada.append(caminho)
                vertice_visitado.add(caminho["estado"].mapeamento)
                if caminho["estado"].profundidade > profundidade_maxima:
                    profundidade_maxima = 1 + profundidade_maxima
        if len(lista_ordenada) > largura_maxima:
            largura_maxima = len(lista_ordenada)

Se for ver bem, ela é similar às demais. É mais fácil pensar que os estados são colocados em uma lista ordenada de acordo com o cálculo da heurística, ou seja, os melhores candidatos são explorados primeiro. 

Vale lembrar que aqui o critério de desempate entre duas possibilidades de próximo estado com heurísticas iguais acontece lá na função de movimento, ou seja, tem a ordem definida por qual movimento acontece primeiro.

A função `ordena_por_prioridade` é uma auxiliar pra fazer a ordenação de uma lista baseada em um argumento.

E isso finaliza nossa implementação, falta apenas ver tudo rodando!

## Execução

Antes de sairmos rodando os algoritmos, vamos definir algumas funções úteis para nos dar os resultados.

Uma função para imprimir o tabuleiro:

In [12]:
def imprime_tabuleiro(estado_atual):
    for linha in range(total_pecas_quebra_cabeca):
        print(estado_atual[linha], sep=' ', end='', flush=True)
        if linha % 3 == 2:
            print('')

E outra pra salvar a profunidade final, o estado final e todos os movimentos realizados até chegar na solução:

In [13]:
def resultados():
    global vertice_solucao
    profundidade = vertice_solucao.profundidade
    estado_final = vertice_solucao.estado
    movimentos = []
    while estado_inicial != vertice_solucao.estado:
        if vertice_solucao.movimento == 1:
            caminho = "CIMA"
        if vertice_solucao.movimento == 2:
            caminho = "BAIXO"
        if vertice_solucao.movimento == 3:
            caminho = "ESQUERDA"
        if vertice_solucao.movimento == 4:
            caminho = "DIREITA"
        movimentos.insert(0, caminho)
        vertice_solucao = vertice_solucao.pai

    print("\n\nRESULTADO:")
    print("###########")
    print("Movimentos:", movimentos)
    print("A resolução tem ", len(movimentos), " passos.")
    print("----------")
    print("Tabuleiro inicial:")
    imprime_tabuleiro(estado_inicial)
    print("----------")
    print("Tabuleiro final:")
    imprime_tabuleiro(estado_final)
    print("----------")
    print("Total de vértices visitados:", str(total_vertices_visitados))
    print("Profundidade da resposta:", str(profundidade))
    print("Profundidade máxima alcancada:", str(profundidade_maxima))
    print("Largura máxima alcancada:", str(largura_maxima))


Como já definimos os estados inicial e final (solução desejada) anteriormente, vamos executar cada busca para ver os resultados.

Começando do começo, a busca em largura:

In [14]:
busca_largura(estado_inicial)
resultados()



RESULTADO:
###########
Movimentos: ['DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'DIREITA']
A resolução tem  8  passos.
----------
Tabuleiro inicial:
012
783
654
----------
Tabuleiro final:
123
804
765
----------
Total de vértices visitados: 257
Profundidade da resposta: 8
Profundidade máxima alcancada: 9
Largura máxima alcancada: 143


Uma boa solução!

E agora, busca em profundidade:

In [15]:
busca_profundidade(estado_inicial)
resultados()



RESULTADO:
###########
Movimentos: ['DIREITA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'ESQUERDA']
A resolução tem  28  passos.
----------
Tabuleiro inicial:
012
783
654
----------
Tabuleiro final:
123
804
765
----------
Total de vértices visitados: 285
Profundidade da resposta: 28
Profundidade máxima alcancada: 28
Largura máxima alcancada: 143


Nada mal!

Agora, busca gulosa com heurística de peças fora do lugar:

In [16]:
busca_gulosa(estado_inicial, '1')
resultados()

 'BAIXO', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'BAIXO', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'BAIXO', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'CIMA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'BAIXO', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQ

É, não foi tão bem...

E com a heurística de distância Manhattan?

In [17]:
busca_gulosa(estado_inicial, '2')
resultados()

EITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'CIMA', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'CIMA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'CIMA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'CIMA', 'CIMA', 'DIREITA', 'BAIXO', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'DIREITA', 'BAIXO', 'DIREITA', 'CIMA', 'ESQUERDA', 'CIMA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'BAIXO', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'DIREITA', 'BAIXO', 'DIREITA', 'CIMA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'CIMA', 'ESQUERDA', 'CIMA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'BAIXO', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA', 'CIMA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'BAIXO', 'ESQUERDA', 'CIMA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'CIMA', 'CIMA', 'DIREITA', 'DIREITA', 'BAIXO', 'ESQUERDA', 'ESQUERDA', 'BAIXO', 'DIREITA', 'DIREITA', 'CIMA

Parece que as buscas cegas levam a melhor!

Por fim, de maneira didática, vimos como resolver um problema da vida real usando algoritmos de busca, a fundação dos sistemas inteligentes e da inteligência artificial.